In [ ]:
from query.datasets.prelude import *
import tempfile

In [ ]:
def extract_audio(video, output_path=None, ext='wav', segment=None):
    if output_path is None:
        output_path = tempfile.NamedTemporaryFile(suffix='.{}'.format(ext), delete=False).name
        
    def fmt_time(t):
        return '{:02d}:{:02d}:{:02d}.{:03d}'.format(
            int(t / 3600), int(t / 60 % 60), int(t % 60), int(t * 1000 % 1000))    
    
    if segment is not None:
        (start, end) = segment
        start_str = '-ss {}'.format(fmt_time(start))
        end_str = '-t {}'.format(fmt_time(end - start))
    else:
        start_str = ''
        end_str = ''
        
    sp.check_call('ffmpeg -y {} -i "{}" {} {}'.format(start_str, video.url(), end_str, output_path), shell=True)
    return output_path

In [ ]:
def parse_segmentation(lines):
    seg = []
    for line in lines:
        if line[:2] == ';;':
            continue
            
        [start, end, gender] = line.split(' ')[2:5]
        seg.append({
            'start': float(start)/100,
            'end': float(end)/100,
            'gender': gender
        })
        
    return seg

def save_segmentation(video, seg):
    labeler, _ = Labeler.objects.get_or_create(name='lium')
    gender_ids = {g.name: g.id for g in Gender.objects.all()}

    speakers = [
        Speaker(video=video, min_frame=int(d['start']*video.fps), max_frame=int((d['start']+d['end'])*video.fps), 
                labeler=labeler, gender_id=gender_ids[d['gender']])
        for d in seg
    ]

    Speaker.objects.bulk_create(speakers)
        
def segment_audio(video, segment=None):
    if Speaker.objects.filter(video=video).count() > 0:
        return
    
    try:
        audio_path = video.extract_audio(segment=segment)
    except sp.CalledProcessError:
        log.error('ffmpeg failed: ' + video.path)
        return
    
    if not os.path.isfile(audio_path):
        log.error('wtf: ' + video.path)
        return

    try:
        seg_path = tempfile.NamedTemporaryFile(delete=False).name
        sp.check_call(
            'java -Xmx4096m -jar /app/deps/LIUM/LIUM_SpkDiarization-4.2.jar --fInputMask={} --sOutputMask={} --doCEClustering _'
            .format(audio_path, seg_path),
            shell=True)
    except sp.CalledProcessError:
        log.error('LIUM failed: ' + video.path)
        return
    finally:
        os.remove(audio_path)
        
    print(seg_path)

#     with open(seg_path) as f:
#         seg_lines = [s.strip() for s in f.readlines()]
#     os.remove(seg_path)
#     seg = parse_segmentation(seg_lines)
    
#     save_segmentation(video, seg)
    
    return seg

In [ ]:
videos = list(Video.objects.all().order_by('?'))

In [ ]:
segment_audio(videos[0])

In [ ]:
# log.debug('Start')
# _ = par_for(segment_audio, videos, workers=80)

In [ ]:
log.debug('Stop')